In [18]:
import tensorflow as tf
import os
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
!aws s3 cp s3://sagemaker-mumbai-data-set data_set --recursive

download: s3://sagemaker-mumbai-data-set/ena-classify-inceptionv3-1-2020-01-14-17-26-47-180/source/sourcedir.tar.gz to data_set/ena-classify-inceptionv3-1-2020-01-14-17-26-47-180/source/sourcedir.tar.gz
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_10093.jpeg to data_set/ena_data/train/air_conditioner/111_10093.jpeg
download: s3://sagemaker-mumbai-data-set/ena-classify-inceptionv3-2-2020-01-14-19-40-13-807/source/sourcedir.tar.gz to data_set/ena-classify-inceptionv3-2-2020-01-14-19-40-13-807/source/sourcedir.tar.gz
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_10013.jpeg to data_set/ena_data/train/air_conditioner/111_10013.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_10124.jpeg to data_set/ena_data/train/air_conditioner/111_10124.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_10205.jpeg to data_set/ena_data/train/air_conditioner/111_10205.jpeg
download: s3:/

download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_12303.jpeg to data_set/ena_data/train/air_conditioner/111_12303.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_11913.jpeg to data_set/ena_data/train/air_conditioner/111_11913.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_11923.jpeg to data_set/ena_data/train/air_conditioner/111_11923.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_11908.jpeg to data_set/ena_data/train/air_conditioner/111_11908.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_12019.jpeg to data_set/ena_data/train/air_conditioner/111_12019.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_122.jpeg to data_set/ena_data/train/air_conditioner/111_122.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_12314.jpeg to data_set/ena_data/train/air_conditioner/111_

download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_13816.jpeg to data_set/ena_data/train/air_conditioner/111_13816.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_14310.jpeg to data_set/ena_data/train/air_conditioner/111_14310.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_13677.jpeg to data_set/ena_data/train/air_conditioner/111_13677.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_14157.jpeg to data_set/ena_data/train/air_conditioner/111_14157.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_14316.jpeg to data_set/ena_data/train/air_conditioner/111_14316.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_13917.jpeg to data_set/ena_data/train/air_conditioner/111_13917.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_14699.jpeg to data_set/ena_data/train/air_conditioner/

download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_16878.jpeg to data_set/ena_data/train/air_conditioner/111_16878.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_17234.jpeg to data_set/ena_data/train/air_conditioner/111_17234.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_16981.jpeg to data_set/ena_data/train/air_conditioner/111_16981.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_16995.jpeg to data_set/ena_data/train/air_conditioner/111_16995.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_1696.jpeg to data_set/ena_data/train/air_conditioner/111_1696.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_17268.jpeg to data_set/ena_data/train/air_conditioner/111_17268.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_16996.jpeg to data_set/ena_data/train/air_conditioner/11

download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_2037.jpeg to data_set/ena_data/train/air_conditioner/111_2037.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_20026.jpeg to data_set/ena_data/train/air_conditioner/111_20026.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_20262.jpeg to data_set/ena_data/train/air_conditioner/111_20262.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_20162.jpeg to data_set/ena_data/train/air_conditioner/111_20162.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_20036.jpeg to data_set/ena_data/train/air_conditioner/111_20036.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_20165.jpeg to data_set/ena_data/train/air_conditioner/111_20165.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/train/air_conditioner/111_19743.jpeg to data_set/ena_data/train/air_conditioner/11

...

download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3141.jpeg to data_set/ena_data/validation/video_game/205_3141.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3149.jpeg to data_set/ena_data/validation/video_game/205_3149.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3139.jpeg to data_set/ena_data/validation/video_game/205_3139.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3144.jpeg to data_set/ena_data/validation/video_game/205_3144.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3178.jpeg to data_set/ena_data/validation/video_game/205_3178.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3186.jpeg to data_set/ena_data/validation/video_game/205_3186.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_32.jpeg to data_set/ena_data/validation/video_game/205_32.jpeg
dow

download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3452.jpeg to data_set/ena_data/validation/video_game/205_3452.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_353.jpeg to data_set/ena_data/validation/video_game/205_353.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3566.jpeg to data_set/ena_data/validation/video_game/205_3566.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3446.jpeg to data_set/ena_data/validation/video_game/205_3446.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3535.jpeg to data_set/ena_data/validation/video_game/205_3535.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3537.jpeg to data_set/ena_data/validation/video_game/205_3537.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/video_game/205_3563.jpeg to data_set/ena_data/validation/video_game/205_3563.jpeg
d

download: s3://sagemaker-mumbai-data-set/ena_data/validation/washing_machine/371_31936.jpeg to data_set/ena_data/validation/washing_machine/371_31936.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/washing_machine/371_32099.jpeg to data_set/ena_data/validation/washing_machine/371_32099.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/washing_machine/371_32221.jpeg to data_set/ena_data/validation/washing_machine/371_32221.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/washing_machine/371_32162.jpeg to data_set/ena_data/validation/washing_machine/371_32162.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/washing_machine/371_32254.jpeg to data_set/ena_data/validation/washing_machine/371_32254.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/washing_machine/371_32257.jpeg to data_set/ena_data/validation/washing_machine/371_32257.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/washing_m

download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_20727.jpeg to data_set/ena_data/validation/water_purifier/111_20727.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/washing_machine/371_33360.jpeg to data_set/ena_data/validation/washing_machine/371_33360.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_20724.jpeg to data_set/ena_data/validation/water_purifier/111_20724.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_20726.jpeg to data_set/ena_data/validation/water_purifier/111_20726.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/washing_machine/371_33473.jpeg to data_set/ena_data/validation/washing_machine/371_33473.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_2083.jpeg to data_set/ena_data/validation/water_purifier/111_2083.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_

download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_2138.jpeg to data_set/ena_data/validation/water_purifier/111_2138.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_21365.jpeg to data_set/ena_data/validation/water_purifier/111_21365.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_21378.jpeg to data_set/ena_data/validation/water_purifier/111_21378.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_21380.jpeg to data_set/ena_data/validation/water_purifier/111_21380.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_21364.jpeg to data_set/ena_data/validation/water_purifier/111_21364.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_21381.jpeg to data_set/ena_data/validation/water_purifier/111_21381.jpeg
download: s3://sagemaker-mumbai-data-set/ena_data/validation/water_purifier/111_2136

In [11]:
!ls data_set/ena_data/

train  validation


In [23]:
IMG_SIZE = 160
BATCH_SIZE = 64

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

INPUT_TENSOR_NAME = "inputs_input"

In [29]:
    base_model = InceptionV3(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
    base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(units=256, activation='relu'))
    model.add(layers.Dense(units=5, activation='softmax'))

In [30]:
tf.keras.models.save_model(
    model,
    '/opt/',
    overwrite=True,
    save_format='tf'
)

NotImplementedError: Saving the model as SavedModel is not supported in TensorFlow 1.Xgraph mode. Please enable eager execution or use the "h5" save format.

In [17]:
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [42]:
model.input

<tf.Tensor 'inception_v3_input_2:0' shape=(?, 160, 160, 3) dtype=float32>

In [19]:
model.model

AttributeError: 'Sequential' object has no attribute 'model'

In [4]:
def keras_model_fn(hyperparameters):
    base_model = InceptionV3(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
    base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(units=256, activation='relu'))
    model.add(layers.Dense(units=5, activation='softmax'))
    
    adamOptimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(optimizer=adamOptimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )
    return model

In [5]:
model = keras_model_fn(hyperparameters=0)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
len(model.trainable_weights)

4

In [27]:
!tensorboard --logdir s3://quikr-datascience-mumbai/image_classify/ena_classify/sagemaker_output/ --host localhost --port 6006


TensorBoard 1.15.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [5]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [24]:
generator = datagen.flow_from_directory('data_set/ena_data/train/', target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)

Found 12600 images belonging to 14 classes.


In [25]:
list(generator.class_indices.keys())

['air_conditioner',
 'air_cooler',
 'camera',
 'desktop',
 'geyser',
 'ipod_mp3_player',
 'laptop',
 'microwave_oven',
 'music_system',
 'refrigerator',
 'tv',
 'video_game',
 'washing_machine',
 'water_purifier']

In [26]:
generator.n // generator.batch_size

196

In [19]:
math.ceil(12600/64)

197

In [21]:
validation_generator = datagen.flow_from_directory(
        'data_set/ena_data/validation', target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)

Found 1400 images belonging to 14 classes.


In [10]:
model.fit_generator(
        generator,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=800)

 99/100 [============================>.] - ETA: 1s - loss: 1.7710 - acc: 0.5487

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 1260s 13s/step - loss: 1.7636 - acc: 0.5493 - val_loss: 1.2828 - val_acc: 0.6979
